In [1]:
%pip install --upgrade pip --quiet
%pip install pyspark --quiet
%pip install -U -q PyDrive --quiet
%pip install numpy pandas --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
SCALA_VERSION = "2.12"
KAFKA_VERSION = "3.7.0"
PYSPARK_SCALA_VERSION = "2.12"
SPARK_VERSION = "3.5.1"

%env SCALA_VERSION=$SCALA_VERSION
%env KAFKA_VERSION=$KAFKA_VERSION
%env PYSPARK_SCALA_VERSION=$PYSPARK_SCALA_VERSION
%env SPARK_VERSION=$SPARK_VERSION

env: SCALA_VERSION=2.12
env: KAFKA_VERSION=3.7.0
env: PYSPARK_SCALA_VERSION=2.12
env: SPARK_VERSION=3.5.1


In [3]:
!bash start-kafka.sh
!tail -n 100 kafka/logs/server.log | grep -i "Kafka Server started"

Starting Kafka 3.7.0 with Scala 2.12
--2024-06-12 08:57:35--  https://dlcdn.apache.org/kafka/3.7.0/kafka_2.12-3.7.0.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 205.251.193.47, 205.251.194.46, 205.251.196.115, ...
Connecting to dlcdn.apache.org (dlcdn.apache.org)|205.251.193.47|:443... failed: Connection timed out.
Connecting to dlcdn.apache.org (dlcdn.apache.org)|205.251.194.46|:443... failed: Connection timed out.
Connecting to dlcdn.apache.org (dlcdn.apache.org)|205.251.196.115|:443... failed: Connection timed out.
Connecting to dlcdn.apache.org (dlcdn.apache.org)|205.251.199.163|:443... failed: Connection timed out.
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 119203328 (114M) [application/x-gzip]
Saving to: ‘kafka_2.12-3.7.0.tgz’

kafka_2.12-3.7.0.tg 100%[===================>] 113.68M  13.7MB/s    in 8.5s    

2024-06-12 09:06:38 (13.4 MB/s) - ‘kafka_2.12-3.7.0.tgz’ saved [11920332

In [4]:
TOPIC1 = "topic1"
TOPIC2 = "topic2"
BOOTSTRAP_SERVER = "127.0.0.1:9092"

In [5]:
!bash create-topics.sh {BOOTSTRAP_SERVER} {TOPIC1} {TOPIC2}
!rm -rf generator.log

Creating kafka topics...
Created topic topic1.
Created topic topic2.


In [6]:
import subprocess

command = [
    "python3", "generator.py",
    "--topic1", TOPIC1,
    "--topic2", TOPIC2,
    "--bootstrap_server", BOOTSTRAP_SERVER
]

GENERATOR = subprocess.Popen(command)
print(f"GENERATOR PID: {GENERATOR.pid}")

GENERATOR PID: 11445


In [7]:
from pyspark.sql import SparkSession

packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{PYSPARK_SCALA_VERSION}:{SPARK_VERSION}',
    f'org.apache.kafka:kafka-clients:{KAFKA_VERSION}'
]

spark = SparkSession.builder \
    .master("local[*]") \
    .config("spark.jars.packages", ",".join(packages)) \
    .appName("PDD task-2") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/12 09:07:01 WARN Utils: Your hostname, DELL-laptop-14-5401 resolves to a loopback address: 127.0.1.1; using 172.20.97.216 instead (on interface eth0)
24/06/12 09:07:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/bartosz/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/bartosz/.ivy2/cache
The jars for the packages stored in: /home/bartosz/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c547f217-a384-4b15-b538-949155ad82d1;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.apache.kafka#kafka-clients;3.7.0 in central
	found com.github.luben#zstd-jni;1.5.5-6 in cent

In [8]:
df = spark.readStream.format("kafka") \
  .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
  .option("subscribe", f"{TOPIC1},{TOPIC2}") \
  .option("startingOffsets", "earliest") \
  .load()

In [9]:
from pyspark.sql.types import *
from pyspark.sql.streaming.state import GroupState
import pandas as pd
import random

SAMPLE_INTERVAL = 100_000
SAMPLE_COUNT = 20
SAMPLE_SIZE = 1_000
LAMBDA = 1e-5
BRS_INSERTION_PROBABILITY = SAMPLE_SIZE * LAMBDA


DATA_SCHEMA = StructType([
    StructField("time_point", IntegerType(), True),
    StructField("value", FloatType(), True)
])

OUTPUT_SCHEMA = StructType([
    StructField("RS_time_point", ArrayType(IntegerType()), False),
    StructField("RS_value", ArrayType(FloatType()), False),
    StructField("BRS_time_point", ArrayType(IntegerType()), False),
    StructField("BRS_value", ArrayType(FloatType()), False),
    StructField("topic", StringType(), False)
])

STATE_SCHEMA = StructType([
    StructField("RS_time_point", ArrayType(IntegerType()), False),
    StructField("RS_value", ArrayType(FloatType()), False),
    StructField("BRS_time_point", ArrayType(IntegerType()), False),
    StructField("BRS_value", ArrayType(FloatType()), False),
    StructField("next_yield", IntegerType(), False)
])

def updateState(_, pdf_iter, state: GroupState):
    (rs_tp, rs_v, brs_tp, brs_v, next_yield_tp) = state.get if state.exists else ([], [], [], [], SAMPLE_INTERVAL)

    for pdf in pdf_iter:
        pdf = pdf.sort_values(by="time_point")
        for _, row in pdf.iterrows():
            row_tp = row["time_point"]
            row_v = row["value"]
            # RS
            if len(rs_tp) < SAMPLE_SIZE:
                rs_tp.append(row_tp)
                rs_v.append(row_v)
            else:
                pos = random.randrange(row_tp)
                if pos < SAMPLE_SIZE:
                    rs_tp[pos] = row_tp
                    rs_v[pos] = row_v
            # BRS
            if random.random() < BRS_INSERTION_PROBABILITY:
                if random.random() < len(brs_tp) / SAMPLE_SIZE:
                    pos = random.randrange(len(brs_tp))
                    brs_tp[pos] = row_tp
                    brs_v[pos] = row_v
                else:
                    brs_tp.append(row_tp)
                    brs_v.append(row_v)
            # YIELD
            if row_tp > next_yield_tp:
                print(f"Topic {row['topic']}: passed {next_yield_tp} timepoint", end="\033[K\n", flush=True)
                next_yield_tp += SAMPLE_INTERVAL
                yield pd.DataFrame({
                    "RS_time_point": [rs_tp.copy()],
                    "RS_value": [rs_v.copy()],
                    "BRS_time_point": [brs_tp.copy()],
                    "BRS_value": [brs_v.copy()],
                    "topic": row["topic"]
                })

    state.update((rs_tp, rs_v, brs_tp, brs_v, next_yield_tp))

In [10]:
import pyspark.sql.functions as F
from pyspark.sql.streaming.state import GroupStateTimeout

sample_dict = {
    TOPIC1: pd.DataFrame(columns=OUTPUT_SCHEMA.fieldNames()),
    TOPIC2: pd.DataFrame(columns=OUTPUT_SCHEMA.fieldNames())
}

def collect_samples(next_sample, _):
    global sample_dict
    next_sample_pdf = next_sample.toPandas()
    for topic in [TOPIC1, TOPIC2]:
        sample_dict[topic] = pd.concat(
            [sample_dict[topic], next_sample_pdf[next_sample_pdf["topic"] == topic]],
            ignore_index=True
        )

processing = df \
    .withColumn("data", F.from_json(F.col("value").cast("string"), DATA_SCHEMA)) \
    .select(
        F.col("data.time_point").alias("time_point"),
        F.col("data.value").alias("value"),
        F.col("topic")
    ) \
    .groupBy("topic") \
    .applyInPandasWithState(
        updateState,
        outputStructType=OUTPUT_SCHEMA,
        stateStructType=STATE_SCHEMA,
        outputMode="append",
        timeoutConf=GroupStateTimeout.NoTimeout
    ) \
    .writeStream \
    .outputMode("append") \
    .foreachBatch(collect_samples) \
    .start()

# Run until enough samples are collected
print("Collecting samples...")
while processing.isActive:
    processing.awaitTermination(5)
    if all(len(sample_pdf) >= SAMPLE_COUNT for sample_pdf in sample_dict.values()):
        print(f"{SAMPLE_COUNT} samples collected from each topic.")
        break

try:
    # Send stop() signal
    print("Sending stop() signal to the processing...")
    processing.stop()

    # Wait for shutdown
    print("Waiting 60sec for the processing to shutdown...")
    processing.awaitTermination(60)

# Shutdown is messy, to say the least...
except Exception:
    pass

24/06/12 09:07:14 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-cb77c0d1-955b-41c0-afef-f2d6c0de0373. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/12 09:07:14 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Topic topic2: passed 100000 timepoint                         (75 + 8) / 200]
Topic topic1: passed 100000 timepoint                         (82 + 8) / 200]
Topic topic2: passed 200000 timepoint=========>              (148 + 8) / 200]
Topic topic1: passed 200000 timepoint==================>     (180 + 8) / 200]
Topic topic2: passed 300000 timepoint                        (106 + 8) / 200]
Topic topic1: passed 300000 timepoint========>               (142 + 8) / 200]
Topic topic2: passed 400000 timepoint                         (76 + 8) / 200]
Topic topic1: passed 400000 timepoint                        (105 + 8) / 200]
Topic topic2: passed 500000 timepoint                         (71 + 8) / 200]
Topic topic1: passed 500000 timepoint                         (99 + 8) / 200]
Topic topic1: passed 600000 timepoint=======================>(199 + 1) / 200]
Topic topic2: passed 600000 timepoint                         (74 + 8) / 200]
Topic topic2: passed 700000 timepoint                         (8

20 samples collected from each topic.
Sending stop() signal to the processing...


24/06/12 09:20:35 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-967a1655-899d-49fa-b694-9b5949bcb98a/37/temp_shuffle_32a3fd62-241d-48ce-a810-55b6edce8676
24/06/12 09:20:35 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-967a1655-899d-49fa-b694-9b5949bcb98a/34/temp_shuffle_1327550a-d537-426a-a2f7-86a2f77de6a4
24/06/12 09:20:35 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-967a1655-899d-49fa-b694-9b5949bcb98a/11/temp_shuffle_6d28bebb-db60-4d85-809e-243555acb343
24/06/12 09:20:35 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-967a1655-899d-49fa-b694-9b5949bcb98a/0c/temp_shuffle_60182f93-ba08-4b1a-8586-af571f26dd6f
24/06/12 09:20:35 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-967a1655-899d-49fa-b694-9b5949bcb98a/20/temp_shuffle_0dc4228f-ed29-41f5-824c-2d7ed2c5ff54
24/06/12 09:20:35 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-967a1655-899d-49fa-b694-9b5949bcb98a/1c/temp_shuffle_8aa62a4b-3f8e-47e2-b0b9-4dcd65f97119
24/06/12 0

Waiting 60sec for the processing to shutdown...


In [11]:
from scipy.stats import kstest

DEPTH_TP = 4

# Trim the samples to have exactly SAMPLE_COUNT
for topic, sample_pdf in sample_dict.items():
    sample_dict[topic] = sample_pdf.drop(columns=["topic"]).head(SAMPLE_COUNT)

print("Kolmogorov-Smirnow test for stream distribution **COMPARISON**\n")
for now_tp in range(SAMPLE_COUNT):
    print(f"now time period: {now_tp * SAMPLE_INTERVAL} - {(now_tp + 1) * SAMPLE_INTERVAL}")
    print(f" RS test result: {kstest(sample_dict[TOPIC1].loc[now_tp,  'RS_value'], sample_dict[TOPIC2].loc[now_tp,  'RS_value'])}")
    print(f"BRS test result: {kstest(sample_dict[TOPIC1].loc[now_tp, 'BRS_value'], sample_dict[TOPIC2].loc[now_tp, 'BRS_value'])}")
    print()

print("\n\n\n")

print("Kolmogorov-Smirnow test for stream distribution **CHANGE**\n")
for now_tp in range(SAMPLE_COUNT):
    for old_tp in reversed(range(max(0, now_tp - DEPTH_TP), now_tp)):
        print(f"old time period: {old_tp * SAMPLE_INTERVAL} - {(old_tp + 1) * SAMPLE_INTERVAL}")
        print(f"now time period: {now_tp * SAMPLE_INTERVAL} - {(now_tp + 1) * SAMPLE_INTERVAL}")
        print(f"Topic: {TOPIC1}")
        print(f"     RS test result: {kstest(sample_dict[TOPIC1].loc[now_tp,  'RS_value'], sample_dict[TOPIC1].loc[old_tp,  'RS_value'])}")
        print(f"    BRS test result: {kstest(sample_dict[TOPIC1].loc[now_tp, 'BRS_value'], sample_dict[TOPIC1].loc[old_tp, 'BRS_value'])}")
        print(f"Topic: {TOPIC2}")
        print(f"     RS test result: {kstest(sample_dict[TOPIC2].loc[now_tp,  'RS_value'], sample_dict[TOPIC2].loc[old_tp,  'RS_value'])}")
        print(f"    BRS test result: {kstest(sample_dict[TOPIC2].loc[now_tp, 'BRS_value'], sample_dict[TOPIC2].loc[old_tp, 'BRS_value'])}")
        print()

Kolmogorov-Smirnow test for stream distribution **COMPARISON**

now time period: 0 - 100000
 RS test result: KstestResult(statistic=0.837, pvalue=0.0, statistic_location=0.9988917112350464, statistic_sign=1)
BRS test result: KstestResult(statistic=0.8293075684380032, pvalue=9.022637152784884e-215, statistic_location=0.9982390999794006, statistic_sign=1)

now time period: 100000 - 200000
 RS test result: KstestResult(statistic=0.849, pvalue=0.0, statistic_location=0.9993849992752075, statistic_sign=1)
BRS test result: KstestResult(statistic=0.8376470588235294, pvalue=4.07484537832928e-300, statistic_location=0.999607264995575, statistic_sign=1)

now time period: 200000 - 300000
 RS test result: KstestResult(statistic=0.839, pvalue=0.0, statistic_location=0.9993849992752075, statistic_sign=1)
BRS test result: KstestResult(statistic=0.8491978609625669, pvalue=0.0, statistic_location=0.999607264995575, statistic_sign=1)

now time period: 300000 - 400000
 RS test result: KstestResult(statis

/home/bartosz/.local/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:394: RuntimeWarning: ks_2samp: Exact calculation unsuccessful. Switching to method=asymp.
  return hypotest_fun_in(*args, **kwds)


In [12]:
for topic, sample_pdf in sample_dict.items():
    print(f"TOPIC: {topic}")
    print(sample_pdf.to_string(max_colwidth=50))
    for id, row in sample_pdf.iterrows():
        print(f"Row {id}:")
        for col in sample_pdf.columns:
            print(f"    Column {col} length: {len(row[col])}")

TOPIC: topic1
                                        RS_time_point                                           RS_value                                     BRS_time_point                                          BRS_value
0   [20416, 63699, 37760, 7323, 65950, 43869, 3469...  [0.7413923740386963, 0.9659909009933472, 0.215...  [56, 120, 35218, 502, 522, 54247, 1794, 659, 7...  [0.6523185968399048, 0.001980634406208992, 0.6...
1   [151528, 110331, 154904, 117792, 65950, 149708...  [0.6538591384887695, 0.5684859156608582, 0.022...  [178371, 120, 115016, 502, 126763, 54247, 1070...  [0.6433680653572083, 0.001980634406208992, 0.9...
2   [243824, 110331, 215260, 117792, 65950, 149708...  [0.15706856548786163, 0.5684859156608582, 0.36...  [178371, 120, 267525, 291459, 204909, 220830, ...  [0.6433680653572083, 0.001980634406208992, 0.1...
3   [243824, 364083, 366838, 391822, 65950, 149708...  [0.15706856548786163, 0.21041226387023926, 0.7...  [358506, 386323, 267525, 291459, 310605, 39207...  [

In [13]:
GENERATOR.terminate()
spark.stop()
!bash kafka/bin/kafka-server-stop.sh